In [47]:
import numpy as np 
import pandas as pd
df = pd.read_csv("D:\downloads\spotify_reccomender_dataset\spotify_millsongdata.csv")
df.head()

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\kanav\AppData\Local\Temp\ipykernel_14652\2173035670.py:3: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv("D:\downloads\spotify_reccomender_dataset\spotify_millsongdata.csv")


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [48]:
df.shape
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [49]:
df = df = df.sample(10000).drop('link', axis=1).reset_index(drop=True)

In [50]:
df.head()

,artist,song,text
0,David Bowie,Janine,"Oh my love, Janine \r\nI'm helpless for your ..."
1,Josh Groban,The Christmas Song,Chestnuts roasting on an open fire \r\nJack F...
2,Moody Blues,Higher And Higher,"Blasting, billowing, bursting forth \r\nWith ..."
3,Vince Gill,Cowboy Up,I'm puttin' on my favorite cowboy boots \r\nL...
4,Natalie Cole,Don't Look Back,Everyday the population is growing \r\nAnd to...


preprocessing

In [51]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n','',regex=True)

In [52]:
import nltk
print(nltk.data.path)

['C:\\Users\\kanav/nltk_data', 'c:\\Users\\kanav\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data', 'c:\\Users\\kanav\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data', 'c:\\Users\\kanav\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data', 'C:\\Users\\kanav\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [53]:
nltk.download('punkt', download_dir='C:/Users/kanav/nltk_data')


[nltk_data] Downloading package punkt to C:/Users/kanav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
nltk.download('punkt')

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kanav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kanav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [56]:

print(tokenization("you are beautiful"))

you are beauti


In [57]:
df.head()

,artist,song,text
0,David Bowie,Janine,"oh my love, janine \ri'm helpless for your sm..."
1,Josh Groban,The Christmas Song,chestnuts roasting on an open fire \rjack fro...
2,Moody Blues,Higher And Higher,"blasting, billowing, bursting forth \rwith th..."
3,Vince Gill,Cowboy Up,i'm puttin' on my favorite cowboy boots \rlik...
4,Natalie Cole,Don't Look Back,everyday the population is growing \rand to f...


In [58]:
df.tail()

,artist,song,text
9995,Kenny Rogers,Scarlet Fever,i stood outside the lucky star \rstaring at t...
9996,Johnny Cash,I Don't Know Where I'm Bound,"i have to sit down for \rok, we'll, we'll do ..."
9997,Bryan White,I'm Not Supposed To Love You Anymore,we agreed that it was over \rnow the lines ha...
9998,Rihanna,If It's Lovin' That You Want,i want to let you know \rthat you don't have ...
9999,Kyla,Cupid,boy if i told you i love you \rthat doesn't m...


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)
similarity[0]

array([1.        , 0.00272536, 0.00515883, ..., 0.01545082, 0.00646054,
       0.01173771])

In [64]:
df[df['song'] == 'The Christmas Song']

,artist,song,text
1,Josh Groban,The Christmas Song,chestnuts roasting on an open fire \rjack fro...
957,Air Supply,The Christmas Song,chestnuts roasting on an open fire \rjack fro...
4326,Owl City,The Christmas Song,"its christmas and we walk alone, \rto strange..."
6360,Garth Brooks,The Christmas Song,chestnuts roasting on an open fire \rjack fro...
8198,Demi Lovato,The Christmas Song,chestnuts roasting on an open fire \rjack fro...


In [67]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:10]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [68]:
recommendation('The Christmas Song')

['The Christmas Song',
 'The Christmas Song',
 'The Christmas Song',
 'Christmas Day',
 'Merry Chistmas Strait To You',
 'Its Christmas',
 'Carol Of The Bells',
 'Have Yourself A Merry Little Christmas',
 'Have Yourself A Merry Little Christmas']

In [69]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))